In [41]:
import requests

# Replace with your actual token
token = "SG_APIM_HMARAVTQFBDNMSBC5XGVGQFP395APFF4MAV2C99FBAAZV6Q6TZ6G"
headers = {
    "Authorization": f"Bearer {token}"
}

def get_relevant_products(query):
    url = f"https://api.sallinggroup.com/v1/product-suggestions/relevant-products?query={query}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        print(f"✅ Found {len(data)} relevant products for query: '{query}'")
        return data
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        return []

relevant_products = get_relevant_products("kylling")

# Loop through the first 5 suggested products
for p in relevant_products.get("products", [])[:5]:
    print(f"- {p.get('name')} (EAN: {p.get('ean')})")



✅ Found 1 relevant products for query: 'kylling'


In [42]:
import json

# Pretty-print the full raw API response
print(json.dumps(relevant_products, indent=2, ensure_ascii=False))


{
  "suggestions": [
    {
      "id": "10210896",
      "prod_id": "92713",
      "title": "Kyllingebrystfilet tilsat 9% vand",
      "description": "Kyllingebrystfilet tilsat 9% vand",
      "img": "https://image.prod.iposeninfra.com/bilkaimg.php?pid=92713&imgType=jpeg",
      "link": "https://www.bilkatogo.dk/p/92713",
      "price": 41.95
    },
    {
      "id": "10553920",
      "prod_id": "156418",
      "title": "Kyllingeinderfilet",
      "description": "Kyllingeinderfilet",
      "img": "https://image.prod.iposeninfra.com/bilkaimg.php?pid=156418&imgType=jpeg",
      "link": "https://www.bilkatogo.dk/p/156418",
      "price": 55
    },
    {
      "id": "10326402",
      "prod_id": "105682",
      "title": "Kyllingebrystfilet",
      "description": "Kyllingebrystfilet",
      "img": "https://image.prod.iposeninfra.com/bilkaimg.php?pid=105682&imgType=jpeg",
      "link": "https://www.bilkatogo.dk/p/105682",
      "price": 179
    },
    {
      "id": "10758562",
      "prod_id"

In [43]:
import requests
import pandas as pd

# Replace with your token
token = "SG_APIM_HMARAVTQFBDNMSBC5XGVGQFP395APFF4MAV2C99FBAAZV6Q6TZ6G"
headers = {
    "Authorization": f"Bearer {token}"
}

# BilkaToGo suggestion search
def get_relevant_bilka_products(query):
    url = f"https://api.sallinggroup.com/v1/product-suggestions/relevant-products?query={query}"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json().get("suggestions", [])
    else:
        print(f"❌ Error {response.status_code}: Failed to fetch for '{query}'")
        return []

# Main function to find cheapest product per ingredient
def find_cheapest_bilka_options(ingredient_list):
    results = []

    for ingredient in ingredient_list:
        suggestions = get_relevant_bilka_products(ingredient)

        if not suggestions:
            results.append({
                "ingredient": ingredient,
                "product_name": "Not found",
                "price": None,
                "link": None
            })
            continue

        # Find the product with the lowest price
        cheapest = min(suggestions, key=lambda x: x.get("price", float('inf')))

        results.append({
            "ingredient": ingredient,
            "product_name": cheapest.get("title"),
            "price": cheapest.get("price"),
            "link": cheapest.get("link")
        })

    return pd.DataFrame(results)


In [44]:
# Example
missing_ingredients = ["mælk", "kyllingebryst", "gulerødder", "løg"]

# lookup
df_cheapest = find_cheapest_bilka_options(missing_ingredients)
print(df_cheapest)

      ingredient           product_name  price  \
0           mælk  Skummetmælk 0,1% fedt  11.50   
1  kyllingebryst     Kyllingebrystfilet  38.95   
2     gulerødder        Snackgulerødder   9.00   
3            løg                 Rødløg   8.00   

                                link  
0   https://www.bilkatogo.dk/p/84122  
1  https://www.bilkatogo.dk/p/115245  
2   https://www.bilkatogo.dk/p/39764  
3  https://www.bilkatogo.dk/p/138030  
